In [1]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import json
import os
from tqdm import tqdm

In [2]:
df = pd.read_csv('Processed/Canadian domestic exports.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.fillna(0, inplace=True)
# df_unpivot = pd.read_csv('Processed/Canadian domestic exports_unpivot.csv')

In [4]:
dates = df.columns.drop(['origin', 'data_type', 'destination', 'industry', 'code_Naics'])
dates = dates.insert(0, dates[-1])
dates = dates.delete(-1)

In [5]:
months = set(dates.map(lambda x: x.split(' ')[1]))

## ydata- profiling

In [10]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("Canadian domestic exports.html")

# find trends 

### xxxxxx codes - Total all countries

In [29]:
df_total = df[df['origin'] == 'Total All Countries']

In [31]:
# calculating mean
df_total['mean'] = df_total[dates].mean(axis=1)

/tmp/ipykernel_333001/1965643245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['mean'] = df_total[months].mean(axis=1)


In [51]:
# finding the rows that have value above mean
above_mean_total = {}
for date in dates:
    greater_than = df_total[date].gt(df_total['mean'])
    above_mean_total[date] = df_total[greater_than][['destination', 'industry', 'code_Naics']]

## xxxxxx codes - subtotal

In [52]:
df_subtotal = df[df['origin'] == 'Sub-Total']
# calculating mean
df_subtotal['mean_sub'] = df_subtotal[date].mean(axis=1)

/tmp/ipykernel_333001/2711053643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subtotal['mean'] = df_subtotal[months].mean(axis=1)


In [53]:
# finding rows with value below mean
below_mean_sub = {}
for date in dates:
    lower_than = df_subtotal[date].lt(df_subtotal['mean_sub'])
    below_mean_sub[date] = df_subtotal[lower_than][['destination', 'industry', 'code_Naics']]

In [54]:
below_mean_sub

{'2021 Jan':           destination                                           industry  \
 7          Washington                      other clothing knitting mills   
 13      Massachusetts     flavouring syrup and concentrate manufacturing   
 19      West Virginia               sanitary paper product manufacturing   
 26               Iowa             plastic plumbing fixture manufacturing   
 36             Nevada                  motor and generator manufacturing   
 ...               ...                                                ...   
 116805    Connecticut       other transportation equipment manufacturing   
 116817   North Dakota  motor vehicle gasoline engine and engine parts...   
 116822      Tennessee                                         apiculture   
 116831     New Mexico           small electrical appliance manufacturing   
 116841     New Mexico  radio and television broadcasting and wireless...   
 
         code_Naics  
 7           315190  
 13          31193

## Comparing

In [63]:
from tqdm import tqdm

In [55]:
# finds the rows with value above mean for all countries and the rows with value below mean for Canada
found_rows = {}
for key in tqdm(above_mean_total.keys()):
    dict_total = above_mean_total[key]
    dict_subtotal = below_mean_sub[key]
    for des in dict_total['destination'].unique():
        for code in dict_total[dict_total['destination'] == des]['code_Naics']:
            temp = dict_subtotal[dict_subtotal['destination'] == des]
            temp = temp[temp['code_Naics']==code]
            columns_list = temp.columns.tolist()
            # create a list of values in each column
            values_list = [temp[column].tolist() for column in columns_list]
            output_list = [[a, b, c] for a, b, c in zip(*values_list)]
            if key not in found_rows.keys():
                found_rows[key] = []
            found_rows[key].append(output_list)


In [57]:
# found rows cotains many empty lists. 
# with this code we remove them
for key in found_rows.keys():
    found_rows[key] = list(filter(lambda x: x, found_rows[key]))

In [58]:
found_rows['2021 Jan'][:4]

[[['Kentucky',
   'butter, cheese, and dry and condensed dairy product manufacturing',
   311515]],
 [['Kentucky', 'breakfast cereal manufacturing', 311230]],
 [['Kentucky', 'oil and gas extraction (except oil sands)', 211110]],
 [['Kentucky', 'dry pea and bean farming', 111130]]]

In [80]:
with open('Processed/rows_with_lower_export.json', 'w') as f:
    json.dump(found_rows, f)

In [75]:
# checking if a phenomenon happened periodicaly
years = ['2021', '2022', '2023']
all_common_structured = {}
for month in months:
    year_month = []
    same_periods = []
    for key in found_rows.keys():
        if month in key:
            same_periods.append(found_rows[key])
    # print(same_periods)
    for e in same_periods[0]:
        # print(e)
        flag = 0
        for lists in same_periods[1:]:
            # for e1 in lists:
            #     print(e1)
            #     print("---")
            if e in lists:
                flag = 1
            else:
                flag = 0
                break
        if flag ==1:
            if e[0][0] not in all_common_structured.keys():
                all_common_structured[e[0][0]] = {}
            if e[0][1] not in all_common_structured[e[0][0]].keys():
                all_common_structured[e[0][0]][e[0][1]] = []
            all_common_structured[e[0][0]][e[0][1]].append(month)
            # print(all_common_structured)
        
    
    

In [79]:
with open('Processed/rows_with_lower_export_frequent_structured.json', 'w') as f:
    json.dump(all_common_structured, f)

# Province wise
we find the destinations and indurstries that have sub-total and Totall all countries values above mean
and then extract the provinces that have export values below mean

In [7]:
df['mean'] = df[dates].mean(axis=1)

In [9]:
a = df[df['origin'].isin(['Total All Countries', 'Sub-Total'])]['2021 Jan'].gt(df['mean'])
b = df[a][['origin','2022 Jan', 'mean', 'destination', 'industry']]
b[(b['destination'] == 'Louisiana')]

,origin,2022 Jan,mean,destination,industry


In [25]:
b.destination.unique()

array(['Kentucky', 'Florida', 'California', 'Georgia', 'Kansas',
       'Indiana', 'Hawaii', 'Arizona', 'Alabama', 'Iowa', 'Alaska',
       'Colorado', 'Arkansas', 'Idaho', 'Connecticut', 'Delaware',
       'District of Columbia', 'Illinois'], dtype=object)

In [12]:
# finding the rows that have value above mean
above_mean_totals = {} #both subtotal and all countries
for date in dates:
    greater_than = df[df['origin'].isin(['Total All Countries', 'Sub-Total'])][date].gt(df['mean'])
    above_mean_totals[date] = df[greater_than][['origin', 'destination', 'industry', 'code_Naics']]

In [ ]:
above_mean_totals['2021 Jan'].destination.unique()

array(['Iowa', 'Nevada', 'Michigan', 'New York', 'Utah', 'Virginia',
       'Kansas', 'Kentucky', 'Nebraska', 'Vermont', 'Oregon', 'Wisconsin',
       'Hawaii', 'Washington', 'California', 'Mississippi', 'Maryland',
       'Texas', 'Indiana', 'Delaware', 'New Hampshire', 'Minnesota',
       'New Jersey', 'Alaska', 'Tennessee', 'Oklahoma', 'Colorado',
       'Florida', 'Massachusetts', 'North Carolina', 'Louisiana',
       'Rhode Island', 'District of Columbia', 'Arizona',
       'South Carolina', 'Montana', 'North Dakota', 'Missouri', 'Maine',
       'Pennsylvania', 'Georgia', 'South Dakota', 'Connecticut',
       'New Mexico', 'Idaho', 'Ohio', 'West Virginia', 'Arkansas',
       'Puerto Rico', 'Alabama', 'Wyoming',
       'Other unspecified U.S. State', 'U.S. Virgin Islands', 'Illinois'],
      dtype=object)

: 

In [29]:
found_rows = {}
for d in tqdm(above_mean_totals.keys()):
    # we find duplicated rows based on values in destination and industry
    both_totals_above_mean = above_mean_totals[d].duplicated(subset=['destination', 'industry'], keep='last')
    df_temp = above_mean_totals[d][both_totals_above_mean]
    # extracting destination and industry and make a list of them
    values_to_match = df_temp[['destination', 'industry']].to_numpy()
    # selecting all rows with specific destination and industry
    selected_rows = pd.DataFrame()
    for values in values_to_match:
        mask = df['destination'].isin(values) & df['industry'].isin(values)
        selected_rows = pd.concat([selected_rows, df[mask]])
    # only select provinces rows
    selected_rows = selected_rows[~selected_rows['origin'].isin(['Sub-Total', 'Sub-total', 'Others', 'Total All Countries'])]
    lower_than = selected_rows[d].lt(selected_rows['mean'])
    found_rows[d] = selected_rows[lower_than][['origin', 'destination', 'industry','mean']]

100%|██████████| 25/25 [21:59<00:00, 52.79s/it]


In [28]:
above_mean_totals.keys()

dict_keys(['2021 Jan', '2021 Feb', '2021 Mar', '2021 Apr', '2021 May', '2021 June', '2021 Jul', '2021 Aug', '2021 Sep', '2021 Oct', '2021 Nov', '2021 Dec', '2022 Jan', '2022 Feb', '2022 Mar', '2022 Apr', '2022 May', '2022 June', '2022 Jul', '2022 Aug', '2022 Sep', '2022 Oct', '2022 Nov', '2022 Dec', '2023 Jan'])

In [21]:
d = '2021 Jan'
both_totals_above_mean = above_mean_totals[d].duplicated(subset=['destination', 'industry'], keep='last')
df_temp = above_mean_totals[d][both_totals_above_mean]
# extracting destination and industry and make a list of them
values_to_match = df_temp[['destination', 'industry']].to_numpy()

In [30]:
c = found_rows['2021 Aug']
c[c['origin'] == 'British Columbia'].destination.unique()

array(['California', 'Louisiana', 'Texas', 'Minnesota', 'Georgia',
       'Washington', 'Florida', 'North Carolina', 'Delaware', 'Michigan',
       'Vermont', 'Montana', 'Iowa', 'Missouri', 'New Jersey',
       'New Hampshire', 'Nevada', 'New York', 'Maryland', 'South Dakota',
       'Tennessee', 'Oklahoma', 'Ohio', 'Alabama', 'South Carolina',
       'Pennsylvania', 'Indiana', 'Massachusetts', 'Virginia',
       'Connecticut', 'North Dakota', 'Kentucky', 'New Mexico',
       'Wisconsin', 'Maine', 'Wyoming', 'West Virginia', 'Oregon',
       'Kansas', 'District of Columbia', 'Colorado', 'Arkansas',
       'Rhode Island', 'Idaho', 'Nebraska', 'Arizona', 'Alaska',
       'Mississippi', 'Other unspecified U.S. State', 'Utah',
       'Puerto Rico', 'Illinois', 'Hawaii'], dtype=object)

In [1]:
'Louisiana' in ['Nevada', 'New York', 'Kentucky', 'Oregon', 'Washington', 'Utah',
       'North Dakota', 'Minnesota', 'Massachusetts', 'Wisconsin',
       'Michigan', 'New Jersey', 'Vermont', 'North Carolina',
       'Pennsylvania', 'Texas', 'Oklahoma', 'Louisiana', 'Nebraska',
       'Ohio', 'Tennessee', 'South Carolina', 'West Virginia',
       'South Dakota', 'Puerto Rico', 'Rhode Island', 'Virginia',
       'New Hampshire', 'Mississippi', 'Montana', 'Wyoming', 'Maine',
       'Missouri', 'Maryland', 'New Mexico',
       'Other unspecified U.S. State']

True

In [31]:
# find rows that happend in all years
# checking if a phenomenon happened periodicaly
years = ['2021', '2022', '2023']
all_common_structured = {} 

for month in months:
    i = 1
    year_month = []
    same_periods = []
    for key in found_rows.keys():
        if month in key:
            same_periods.append(found_rows[key])
    
    if month == 'Jan':
        s1 = pd.merge(same_periods[0], same_periods[1], how='inner', on=['origin', 'destination', 'industry'])
        s2 = pd.merge(same_periods[1], same_periods[2], how='inner', on=['origin', 'destination', 'industry'])
        print(s1)
        if len(s1)< len(s2):
            s1 = s2
            i = 3
    else:
        i = 1
        s1 = same_periods[0]
    try:
        for df1 in same_periods[i:]:
            s1 = pd.merge(s1, df1, how='inner', on=['origin', 'destination', 'industry'])
    except:
        pass
        
    # making a structured dictionary
    for e in s1.to_numpy():
        if e[0] not in all_common_structured.keys():
            all_common_structured[e[0]] = {}
        if e[1] not in all_common_structured[e[0]].keys():
            all_common_structured[e[0]][e[1]] = {}
        if e[2] not in all_common_structured[e[0]][e[1]].keys():
            all_common_structured[e[0]][e[1]][e[2]] = []
        all_common_structured[e[0]][e[1]][e[2]].append(month)
    


                       origin  destination  \
0                 Nova Scotia         Iowa   
1                     Ontario         Iowa   
2                Saskatchewan      Arizona   
3                    Manitoba      Arizona   
4                     Ontario      Florida   
5                    Manitoba      Florida   
6            British Columbia      Florida   
7                      Quebec     Illinois   
8        Prince Edward Island     Illinois   
9                      Quebec        Idaho   
10           British Columbia        Idaho   
11               Saskatchewan         Iowa   
12           British Columbia         Iowa   
13                     Quebec         Iowa   
14                    Alberta      Indiana   
15               Saskatchewan      Indiana   
16              New Brunswick      Indiana   
17                Nova Scotia      Indiana   
18                    Ontario     Illinois   
19           British Columbia     Illinois   
20                    Ontario     

In [32]:

with open('Processed/rows_with_lower_export_frequent_structured_provinces.json', 'w') as f:
    json.dump(all_common_structured, f)

In [33]:
all_common_structured['British Columbia']['Louisiana']

{'all other textile product mills': ['June'],
 'structural wood product manufacturing': ['June'],
 'primary production of alumina and aluminum': ['June', 'Aug'],
 'cutlery and hand tool manufacturing': ['June'],
 'jewellery and silverware manufacturing': ['Mar'],
 'copper rolling, drawing, extruding and alloying': ['Mar', 'Jan'],
 'nonwoven fabric mills': ['Mar'],
 'plastic bottle manufacturing': ['Mar'],
 'rubber and plastics industry machinery manufacturing': ['Mar'],
 'clay building material and refractory manufacturing': ['Mar'],
 'sporting and athletic goods manufacturing': ['Mar'],
 'all other food manufacturing': ['Mar'],
 'seasoning and dressing manufacturing': ['Mar'],
 'clothing accessories and other clothing manufacturing': ['Oct'],
 'all other miscellaneous wood product manufacturing': ['Oct'],
 'polystyrene foam product manufacturing': ['Oct'],
 'confectionery manufacturing from purchased chocolate': ['Oct', 'Dec', 'Nov'],
 'chocolate and chocolate confectionery manufactur